## read data from files netatmo

In [45]:
import numpy as np
#import tensorflow as tf
#from tensorflow import keras
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters


#!pip install wget 
import wget
#wget.download('https://raw.githubusercontent.com/BorisMuzellec/MissingDataOT/master/utils.py')

import numpy as np
import pandas as pd
from utils import *
import torch
import seaborn as sns

from sklearn.model_selection import cross_val_score

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

In [116]:
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
data=pd.read_csv('dataCompletAvrilto18Juin.csv', sep=";", encoding = "ISO-8859-1", index_col=index_col)
#df.to_csv('MCAR25.csv')
# supprimer d'abord les deux lignes dans le fichier d'export Neatatmo
index_col="time"
p="10.csv"

df=pd.read_csv('MCAR'+p, sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=pd.read_csv('mask'+p, sep=",", encoding = "ISO-8859-1", index_col=index_col)
mask=mask.values



In [ ]:
X_kalman=pd.read_csv('MCAR'+p, sep=",", encoding = "ISO-8859-1")
X_kalmanArimaImputed=pd.read_csv('kalmanArima_TRUE_MCAR'+p, sep=",", encoding = "ISO-8859-1")
X_kalmanStructTsImputed=pd.read_csv('kalmanStructTS_TRUE_MCAR'+p, sep=",", encoding = "ISO-8859-1")
X_kalmanArimaImputed=X_kalmanArimaImputed.set_index('time')
X_kalmanStructTsImputed=X_kalmanStructTsImputed.set_index('time')

In [117]:
# Fix the seed ------------------------------------------------------
np.random.seed(0)

In [118]:
X = data.copy()
X = X.drop(["activity"], axis=1)
X = X.values


In [119]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
datascaler=scaler.fit(X)
X= datascaler.transform(X)

## Time Series Imputation Techniques

In [203]:
### Forward Fill
# Impute airquality DataFrame with ffill method

### Forward Fill
# Impute airquality DataFrame with ffill method
import numpy as np
from sklearn.impute import KNNImputer
X1 = df.copy()
# add time "timpstame" as variable 
X1["time"]=X1.index
k=2
imputer = KNNImputer(n_neighbors=k)
KNNimputed = imputer.fit_transform(X1)
KNNimputed=KNNimputed[:,0:-1].copy()
KNNimputed= pd.DataFrame(data=KNNimputed, columns=df.columns)
KNNimputed.index = df.index

ffill_imputed = df.copy(deep=True)
ffill_imputed.fillna(method='ffill',inplace=True)
ffill_imputed.fillna(method='bfill',inplace=True)

bfill_imputed = df.copy(deep=True)
bfill_imputed.fillna(method='bfill',inplace=True)
bfill_imputed.fillna(method='ffill',inplace=True)

quadratic_imput = df.copy(deep=True)
quadratic_imput.interpolate(method='quadratic', inplace=True)
quadratic_imput.fillna(method='ffill',inplace=True)
quadratic_imput.fillna(method='bfill',inplace=True)


nearest_imput = df.copy(deep=True)
nearest_imput.interpolate(method='nearest', inplace=True)
nearest_imput.fillna(method='ffill',inplace=True)
nearest_imput.fillna(method='bfill',inplace=True)


linear_imput=df.copy(deep=True)
linear_imput.interpolate(method='linear', inplace=True)
linear_imput.fillna(method='ffill',inplace=True)
linear_imput.fillna(method='bfill',inplace=True)

poly_imput=df.copy(deep=True)
poly_imput= poly_imput.interpolate(method='polynomial', order=3)
poly_imput.fillna(method='ffill',inplace=True)
poly_imput.fillna(method='bfill',inplace=True)




# Save dataFrame Imputed

In [115]:
# Create a dictionary of interpolations
# interpolations = {'Back_fill':bfill_imputed, 'Forward_fill':ffill_imputed,
#                   'Linear_Interpolation': linear_imput, 'Quadratic_Interpolation': quadratic_imput,"Nearest_imput":nearest_imput, "polynomial_interpolation":poly_imput, 'KNN': KNNimputed}
# # enregistrer les données imputéés par chaque méthode dans le dossier dataImputed
interpolations = {'meanImputation':mean_imput}
for df_key in  interpolations:
    print(df_key+p,type(interpolations[df_key]))
    interpolations[df_key].to_csv('dataImputed/'+df_key+p)
            

    

meanImputation5.csv <class 'pandas.core.frame.DataFrame'>


In [ ]:
mean_imput = df.copy(deep=True)
mean_imput =mean_imput.fillna(mean_imput.mean())
mean_imput

In [ ]:
mean_imput

In [4]:
df.shape

(22523, 5)

In [ ]:
import numpy as np
from sklearn.impute import KNNImputer
X1 = ff.copy()
#X1["time"]=X1.index
print(X1.columns)
for k in range(10,20):
    imputer = KNNImputer(n_neighbors=k)
    KNNimputed = imputer.fit_transform(X1)
    Ximp= KNNimputed    
    Ximp = Ximp[:,0:-1]
    print("k = ",k)
    for i in range(5):
        print("KNN", "  "+str(df.columns[i]), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
    print("KNN", " all ", RMSE(Ximp[:,0:5], X,mask))

In [121]:

Ximp= mean_imput.values   
#Ximp = Ximp[:,0:-1]
for i in range(5):
    print("mean", "  "+str(i), RMSE(Ximp[:,i], X[:,i], mask[:,i]))
print("mean", " all ", RMSE(Ximp, X,mask))

mean   0 1.0280782741512864
mean   1 1.0039951888402086
mean   2 0.9988984776443648
mean   3 1.0438501895091625
mean   4 1.0267607456125483
mean  all  1.0203231350042317


time
1617235589    0.369908
1617235891    0.000948
1617236194    0.369908
1617236497    0.000948
1617236799    0.369908
                ...   
1624059673    1.149909
1624059976    1.149909
1624060279    1.149909
1624060581    0.000948
1624060883    1.149909
Name: Humidity, Length: 22523, dtype: float64

In [75]:
mask.shape

(22523, 5)

In [ ]:
Ximp

In [180]:
import numpy as np
from sklearn.impute import KNNImputer
X1 = df.copy()
k=1100
imputer = KNNImputer(n_neighbors=k)
KNNimputed = imputer.fit_transform(X1)


In [15]:
def lagged(df,lag=2):
    #df=df.set_index('time')    
    Df=[]
    a=[]
    for i in range(lag):
        a.append(i)
        df1=df.shift(i+1).copy()
        df1=df1.add_suffix('_t-'+str(i+1))
        Df.append(df1)
    for base in Df:
        df=pd.concat([df,base], axis=1)
    
    #df=df.drop(df.index[a])
    return df, a 

In [38]:
# iterative imputation transform for the horse colic dataset
from numpy import isnan
from pandas import read_csv
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
# load dataset
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/horse-colic.csv'
dataframe = read_csv(url, header=None, na_values='?')
# split into input and output elements
data = dataframe.values
ix = [i for i in range(data.shape[1]) if i != 23]
X, y = data[:, ix], data[:, 23]
# print total missing
print('Missing: %d' % sum(isnan(X).flatten()))
# define imputer
imputer = IterativeImputer()
# fit on the dataset
imputer.fit(ff)
# transform the dataset
Xtrans = imputer.transform(ff)
# print total missing
print('Missing: %d' % sum(isnan(Xtrans).flatten()))

Missing: 1605
Missing: 0


In [54]:
#!pip install autoimpute

In [58]:
a

<generator object MiceImputer.transform.<locals>.<genexpr> at 0x00000207E3557270>